In [ ]:
# from your repo root
python graph/compare.py \
  --root data/results/Qubits8 \
  --out  graph/Qubits8 \
  --ema  0.9 \
  --metrics loss mmd kl grad_norm step_time
0.7
Step 999: loss=0.075449 mmd=0.002077 kl=0.246650 ||g||=0.001167 dt_ms=0.029ms
0.1
Step 999: loss=0.224564 mmd=0.002195 kl=0.249271 ||g||=0.003662 dt_ms=0.127ms
0.2
Step 999: loss=0.211111 mmd=0.002905 kl=0.263162 ||g||=0.005027 dt_ms=0.029ms
0.3
Step 999: loss=0.172192 mmd=0.002165 kl=0.245061 ||g||=0.003197 dt_ms=0.157ms
0.4
Step 999: loss=0.148880 mmd=0.002176 kl=0.246683 ||g||=0.002398 dt_ms=0.156ms
0.5
Step 999: loss=0.124427 mmd=0.002158 kl=0.246695 ||g||=0.002026 dt_ms=0.128ms

usage: ipykernel_launcher.py [-h] --root ROOT --out OUT [--runs [RUNS ...]]
                             [--labels [LABELS ...]] [--ema EMA]
                             [--metrics [METRICS ...]]
ipykernel_launcher.py: error: the following arguments are required: --root, --out


SystemExit: 2

/home/cx/miniforge3/envs/gg/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
